<a href="https://colab.research.google.com/github/mraskj/css_fall2023/blob/main/code/class10/class10-solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class 10: Audio Measurement - Solution

In this exercise, we explore how we can use a Python-binding of Praat, a speech analysis software designed to study phonetics (the sound of speeches) using computers, to study acoustic features of political speeches.



## 0 Setup

We start by:

1. Cloning the course GitHub repo
2. Install necessary packages
3. Importing necessary modules
4. Reading data




### 0.1 Cloning GitHub Repository

In [ ]:
# Clone GitHub directory into
!git clone https://github.com/mraskj/css_fall2023.git

### 0.2 Install Packages

In [ ]:
# Install packages
!pip install -r /content/css_fall2023/requirements/requirements_topic4-class9.txt

### 0.3 Importing Modules

In [ ]:
# For file and directory management
import os

# For regular expressions
import re

# For data handling
import numpy as np
import pandas as pd

# For plotting
import seaborn as sns
import matplotlib.pyplot as plt
sns.set() # Use seaborn's default style
# Feature extraction
import librosa

# For Praat
import parselmouth

### 0.4 Reading Data

In this exercise, we will work with 20 audio files each corresponding to a speech in the Danish parliament in the second assembly of the 2007 parliamentary term (*20072*). The files come from 10 different speakers with two speeches each. FIve of the speakers are men and Five are women.

You find the files in *data/audio/class10/* folder in the cloned GitHub repo.

In [ ]:
# Define path to directory containing audio files
base_dir = os.path.join(os.getcwd(), 'css_fall2023/data/audio/class10')

male_files = os.listdir(os.path.join(base_dir, 'male'))
female_files = os.listdir(os.path.join(base_dir, 'female'))

If we print `male_files` and `female_files`, we'll see that they are unsorted. To keep things in order, we sort the files by the first digits in the filenames. We define a custom function for this purpose and provide the function as parameter to the `key` argument in the `sorted()` function.

In [ ]:
# Define a custom sorting key function
def digit_sort_key(v):
    digits = re.search('\d+', v)
    return int(digits.group())

# Sort the lists
male_files = sorted(male_files, key=digit_sort_key)
female_files = sorted(female_files, key=digit_sort_key)

Each audio file corresponds to speech given in a parliamentary debate in the Danish Parliament. In the GitHub repo, you find a dataset with metadata for each file: `class10_audio_df.csv`. Read in the data to an object called `speech_df`. It contains metadata for the audio files such as the name of the file, the speaker, and a bunch of other variables. Before we proceed, we divide `speech_df`, into two: `female_speech_df` and `male_speech_df`.

In [ ]:
# Read in dataframes
speech_df = pd.read_csv('/content/css_fall2023/data/class10_audio_df.csv')

# Divide dataset into two - note that we can also filter the data based on the `gender` column
female_speech_df = speech_df.iloc[:10]
male_speech_df = speech_df.iloc[10:]

## Exercise 1: Visualization of Pitch

While `parselmouth-praat` enables us to measure a wide range of acoustic features, we focus on the fundamental frequency ($F0$) here. Feel free to play around with others as well.

Recall from the lecture that $F0$ refers to the number of times per second our vocal cords vibrate when making voiced sounds. As the name suggests, voiced sounds refer to sounds that make use of our vocal cords. All vowel sounds are voiced, but not all consonants are. Some consonant sounds, such as /p/, /f/ and /s/ are unvoiced. This means the vocal cords do not vibrate when producing these sounds.

For our purpose, we are talking about $F0$ at a higher level than vowels and consonants, typically at the speech level, but other semantic units such as sentences are also likely or fixed temporal segments (e.g. a five second window). Regardless of the level of measurement, the fundamental frequency is called the *fundamental* since it's the dominant frequency in the given time period that we consider.

As we saw on the slides, the $F0$ is a strong and robust indicator of emotional arousal. The idea is simple. When a speaker becomes emotional aroused, or if a speaker wants to *project* arousal, $F0$ increases. The link is not necessarily causal, but what matters to us is that arousal is reflected in $F0$. We will explore this idea in this exercise by estimating the pitch of speeches.

The fundamental frequency of a sound is closely correlated with pitch. Although some may think that $F0$ is just a fancy word for pitch, others argue that pitch is a different concept. In fact, pitch is more to do with a person's perception of a $F0$. In other words, fundamental frequency refers to the actual physical properties of a sound wave signal, whereas pitch refers to how our ears and brains perceive the signal regarding the rate of frequency.

In this exercise, we will investigate whether we can interpret a speaker's level of emotional arousal by computing $F0$ for the 20 audio files and projecting the estimates onto a spectrogram. We did something similar in the tutorial. Your task is to:


>  **Plot each speech as a spectrogram, compute $F0$ and project it onto the spectrogram**.

1. Make two plots, one of female speakers and one for male speakers. Arrange the plots in a $5\times 2$ grid with each row being a single speaker.
2. Describe your modeling choices.

    a) How do you handle pitch estimates that are zero?

    b) Which settings do you use to compute the pitch?

    c) How do you control the time range? (see step 3)

    d) Do you restrict the y-axis of the spectrogram? Can be done by explictly passing a value to the `maximum_frequency` argument in the `parselmouth.Sound.to_spectrogram()` method.
3. Conduct step 1 for different time ranges and describe the differences.

    - Range 1: Full speech
    - Range 2: 15 seconds of speech
    - Range 3: 5 seconds of speech
    - Range 4: 2 seconds of speech



4. Based on the results, can you visually identify the speech where the speaker is most aroused? Why, why not? Does the time range matters?

## Solution 1

In [ ]:
def draw_spectrogram(spectrogram, start=0, stop=10000, dynamic_range=70, cmap='afmhot'):
    """
    Draw a spectrogram using Matplotlib.

    Parameters:
    spectrogram (Spectrogram): The input spectrogram.
    start (float, optional): The start time in milliseconds. Defaults to 0.
    stop (float, optional): The stop time in milliseconds. Defaults to 10000.
    dynamic_range (float, optional): The dynamic range in decibels. Defaults to 70.
    cmap (str, optional): The colormap to use. Defaults to 'afmhot'.

    Returns:
    None
    """
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    indices = np.where((X >= start) & (X <= stop))[0]
    X = X[indices]
    sg_db = 10 * np.log10(spectrogram.values)
    sg_db = sg_db[:,indices[:-1]]
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap=cmap)
    plt.ylim([spectrogram.ymin, spectrogram.ymax])


def draw_pitch(pitch):
    """
    Draw a pitch contour using Matplotlib.

    Parameters:
    pitch (Pitch): The input pitch object.

    Returns:
    Tuple[float, float]: A tuple containing the mean F0 and standard deviation of F0.
    """
    pitch_values = pitch.selected_array['frequency']
    f0mean = np.mean(pitch_values[pitch_values > 0])
    f0std = np.std(pitch_values[pitch_values > 0])
    pitch_values[pitch_values==0] = np.nan

    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='#381a61')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2, color='#e78429')

    plt.grid(False)
    plt.ylim(0, pitch.ceiling)

    return np.round(f0mean, 2), np.round(f0std, 2)


def draw_spectrogram_pitch(files:list,
                           gender:str,
                           dataframe,
                           start=None,
                           stop=None,
                           max_freq = 6000,
                           cmap='afmhot',
                           show=True):


    """
    Draw spectrograms and pitch contours for a list of audio files.

    Parameters:
    files (list): List of audio files.
    gender (str): Gender of the speaker.
    dataframe (DataFrame): Dataframe containing speaker information.
    start (float, optional): The start time in seconds. If not provided, it's determined from the audio data. Defaults to None.
    stop (float, optional): The stop time in seconds. If not provided, it's determined from the audio data. Defaults to None.
    max_freq (float, optional): The maximum frequency for the spectrogram. Defaults to 6000.
    cmap (str, optional): The colormap to use. Defaults to 'afmhot'.
    show (bool, optional): Whether to display the plots. Defaults to True.

    Returns:
    None
    """
    n_plots = len(files)
    num_rows = (n_plots + 1) // 2
    num_cols = n_plots // num_rows

    plt.figure(figsize=(30, 22))

    for c, f in enumerate(files):
        snd = parselmouth.Sound(os.path.join(base_dir, gender, files[c]))

        if not start and not stop:
          start, stop = int(snd.xmin), int(snd.xmax)

        pitch = snd.to_pitch()
        spectrogram = snd.to_spectrogram(window_length=0.025, maximum_frequency=max_freq)

        plt.subplot(num_rows, num_cols, c + 1)

        draw_spectrogram(spectrogram, start=start, stop=stop, cmap=cmap)

        if c % 2 == 0:
          plt.ylabel('Frequency (Hz)')

        if c % 2 == 1:
          plt.yticks([])

        if np.abs(c - n_plots) <= 2:
          plt.xlabel('Time (s)')

        plt.twinx()
        f0mean, f0std = draw_pitch(pitch)

        if c % 2 == 0:
          plt.yticks([])

        if c % 2 == 1:
          plt.ylabel('F0 (Hz)')

        if c < 8:
          plt.xticks([])

        plt.xlim([start, stop])
        plt.title(f"Speaker: {dataframe.iloc[c]['sppid']} (F0 mean={np.round(f0mean, 3)}, F0 std={np.round(f0std, 3)})")

    if show:
      plt.show()


In [ ]:
# Draw plot for male speakers
draw_spectrogram_pitch(files=male_files,
                       gender='male',
                       dataframe=male_speech_df,
                       cmap='viridis',
                       start=20,
                       stop=25)

In [ ]:
# Draw plot for female speakers
draw_spectrogram_pitch(files=female_files,
                       gender='female',
                       dataframe=female_speech_df,
                       cmap='viridis',
                       start=5,
                       stop=20)

## Exercise 2: Standardized vs Unstandardized Pitch

In *Exercise 1*, we visualized pitch contours across the $20$ speeches comparing two speeches for each speaker at a time. This pairwise-comparison indirectly parses out any speaker heterogeneity since we compare speeches within the same speaker.

To see why this is important, we will now compute the speech-level average of $F0$ for each speech to investigate the impact of speaker heterogeneity when interpreting results.  



> **Task 1: Compute the speech-level average $F0$ for both male and female speakers. Add metadata from the `speech_df` and combine into a dataframe. Take the columns 'sppid', 'gender' and 'spitch'. The 'sppid' column is the speaker identifier containing the name of the speaker and the party. The 'gender' column is the gender of the speaker. And the `spitch' column contains standardized pitch estimates for that specific speech computed over speeches from 2000-2021. Hence, hundreds and even thousands of speeches contribute to the score. Recall that standardization scales data to mean zero and unit variance. Sort the computed F0 means from highest to lowest and describe the results. Pay attention to the difference between your F0 estimates and the standardized estimates.**

> **Task 2: Plot the pitch estimates for the first speech for each speaker of your $20$ files. Make a total of three plots. The first plot should contain pitch distributions for both male and female speakers in one. The second plot should contain pitch distributions for female speakers only. And the third plot should contain pitch distributions for male speakers only. Arrange the plot in a grid where you have the second and third plot vertically stacked. Describe the results.**






### Solution Task 1

In [ ]:
# Compute the pitch for all 20 speeches.
# I iterate over the files in `female_files` first and then `male_files`.
# For each loop, I:
#     - load the file using `parselmouth.Sound()`
#     - convert the sound object to a pitch object using `.to_pitch()`
#     - extract the values using `.selected_array['frequency']`
#     - save all non-zero estimates in an empty list associated with each loop

female_pitch_list = []
for c, f in enumerate(female_files):
    snd = parselmouth.Sound(os.path.join(base_dir, 'female', female_files[c]))
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    female_pitch_list.append(pitch_values[pitch_values > 0])

male_pitch_list = []
for c, f in enumerate(male_files):
    snd = parselmouth.Sound(os.path.join(base_dir, 'male', male_files[c]))
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    male_pitch_list.append(pitch_values[pitch_values > 0])

In [ ]:
# Compute the average F0 for each file and store the results in two lists.
# Note that I use a zip() with the filenames to make the link
# between the values and the audio files explicit.
male_f0mean = [(x, y) for x, y in zip(male_files, [np.mean(x) for x in male_pitch_list])]
female_f0mean = [(x, y) for x, y in zip(female_files, [np.mean(x) for x in female_pitch_list])]

# Unpack the list of tuples into two separate tuples
file_list, f0_list = zip(*female_f0mean + male_f0mean)

In [ ]:
# Construct a dataframe based on the `speech_df` and the two separate tuples.
# Note that the tuples must be converted to a pd.Series or pd.DataFrame object
# to be used in the pd.concat() function. I specify axis=1 to say that I want
# to concatenate at the column-level.
f0_df = pd.concat([speech_df[['sppid', 'gender', 'spitch']],
           pd.Series(file_list, name='file'),
           pd.Series(f0_list, name='f0')],
          axis=1)

In [ ]:
# Sort values by the 'f0' column in descending order
f0_df = f0_df.sort_values(by='f0', ascending=False)

In [ ]:
# Correlate spitch and f0 - correlation is 0.305 indicating only a weak association
f0_df[['spitch', 'f0']].corr()

### Solution Task 2

In [ ]:
# Compute mean pitch for men and women -- use [::2] indexing to extract every second speech
male_mean = np.mean([np.mean(x) for x in male_pitch_list[::2]])
female_mean = np.mean([np.mean(x) for x in female_pitch_list[::2]])

In [ ]:
# Plot distribution of pitch for men and women arranged in a grid.
plt.figure(figsize=(14,8))

plt.subplot(2, 2, 1)
ax = sns.kdeplot(male_pitch_list[::2] + female_pitch_list[::2], legend=False)
ax.text(male_mean + 10, 0.00125, '$\u2642$')
ax.text(female_mean + 10, 0.00125, '$\u2640$')
plt.axvline(male_mean, 0, 5, linestyle='--', color='black')
plt.axvline(female_mean, 0, 5, linestyle=':', color='black')
plt.ylabel('')
plt.yticks([])
plt.xlabel('Hz')

plt.subplot(2, 2, 2)
sns.kdeplot(female_pitch_list[::2])
plt.legend(labels=list(female_speech_df.iloc[::2].sppid.values), frameon=False,fontsize=10)
plt.ylabel('')
plt.yticks([])

plt.subplot(2, 2, 4)
sns.kdeplot(male_pitch_list[::2])
plt.legend(labels=list(male_speech_df.iloc[::2].sppid.values), frameon=False, fontsize=10)
plt.ylabel('')
plt.yticks([])
plt.show()